# Running LocusCompare
After obtaining a hit, you check the co-localization (with dose-dependent signal strength) between the GWAS results and eQTLs. Use the LocusCompare.    
* [Abundant associations with gene expression complicate GWAS follow-up, Nature Genetics](https://www.nature.com/articles/s41588-019-0404-0) Boxiang Liu, Michael J. Gloudemans, Abhiram S. Rao, Erik Ingelsson & Stephen B. Montgomery (2019) 

The R versin of the software readme is [here](https://github.com/boxiangliu/locuscomparer)

# Input file
For LocusCompare we only need:

    rsid	pval    
    rs62156064	0.564395    
    rs7562234	0.399642    
    rs11677377	0.34308    
    rs35076156	0.625237

Basic pipeline

1. Decide the slicing range
2. Slice the relevant locus for two summary stats
3. Format the data
4. Run the analysis

We can download the summary statistics from LocusZoom. This has been annotated with RSID conveniently.

In [1]:
# Preparation
import glob
import os
import subprocess
import pandas as pd
import subprocess
import shutil
import time
import numpy as np
import sys
from IPython.core.debugger import set_trace # insert 'set_truce()' inside function for debagging
import matplotlib as plt

# Some job control commands
def submitMultiJobs(x, time='3:00:00', mem=2, threads=1, bundle=1, modules=None, store='swarm', node='quick', packing=True, go=False):
    """
    Submit multiple jobs on biowulf
    x: LIST of the location of swarmfile
    max runtime: quick->4:00:00, norm->48:00:00 be careful when bundle jobs
    node: quick or norm
    example of modules: modeule='R,plink'
    'go=True' will submit the jobs
    """
    swarmdevq="swarm -f {F} --time={T} -g {G} -p {PACK} -b {B} -t {THREADS} --logdir {L}{M} --partition={P} --devel"
    a=[]
    MODULES=' --module {M}'.format(M=modules)*(modules is not None)
    for i in x: 
        s=swarmdevq.format(F=i, T=time, G=mem, PACK=1+packing, B=bundle, M=MODULES, 
                           THREADS=threads, L=store, P=node)
        if go:
            res=subprocess.run(s[:-8].split(" "), stdout=subprocess.PIPE)
            a.append(res.stdout.decode('utf-8'))
        if not go:
            res=subprocess.run(s.split(" "), stdout=subprocess.PIPE)
            t = res.stdout.decode('utf-8').replace(',', '--').replace('\n', '--').replace(' /spi', '--').split("--")
            x = [i for i,x in enumerate(t) if 'cpus-' in x]
            print(t[1:3]+t[x[0]:-2])
            a='Check!![module, N_jobs, node, mem(x2), and store,bundle] If ok -> go=True!'
    return(a)


def checkJobStatus(list_of_jobids):
    """
    Get the job status as the list of dataframe with some outputs
    """
    ans = []
    index=0
    for i in list_of_jobids:
        res = subprocess.run(['jobhist', str(i)], stdout=subprocess.PIPE)
        res2 = res.stdout.decode().split('\n')
        start = next(i for i,x in enumerate(res2) if 'Jobid' in x)
        start_script = next(i for i,x in enumerate(res2) if 'Swarm Command' in x)+2
        script = res2[start_script].split('-f')[1]
        s = pd.Series(res2[start:])
        df = s.str.split("\\s{2,}", expand=True)
        df = df.rename(columns=df.iloc[0, :])
        a=df[1:-1]
        print('=========== index=' + str(index) + ": jobID=" + i + ": " + script + ' ===============')
        print("N of Jobs: " + str(a.shape[0]))
        print('N_of not_COMPLETED: '+str(sum(a.State!='COMPLETED')))
        print("First 5 of longest Runtime job")
        print(a.sort_values('Runtime', ascending=False).head())
        print("Pending jobs"+str(list(a[a.State=='PENDING'].Jobid)))
        ans.append(a)
        index += 1
    return (ans)

def plotRuntime(Runtime):
    """
    Create plot from Outputted Runtime from checkJobStatus
    """
    def convertH(runtime):
        h, m, s = runtime.split(':')
        return int(h) + int(m) / 60 + int(s)/3600
    Runtime.apply(convertH).plot.hist()

def checkSubjob(subjobID):
    """
    Get the command of subjobs to check. 
    Provide subjobID in string
    """
    header='/spin1/swarm/iwakih2/'
    findfolder=header+subjobID.replace("_", '/cmd.')
    files = glob.glob(findfolder+'_*')
    for file in files:
        res = subprocess.run(['cat', file], stdout=subprocess.PIPE)
        return(print(res.stdout.decode()))

def retrieveTimeouts(x, output):
    """
    Writing timeouted jobs in 'output'
    x is the output dataframe from "checkJobStatus"
    """
    failed=x[x.State=='TIMEOUT'].Jobid
    header='/spin1/swarm/iwakih2/'
    findlist=[header+i.replace("_", '/cmd.') for i in failed]
    with open(output, 'w') as f:
        for i in findlist:
            files = glob.glob(i+'_*')
            for file in files:
                res = subprocess.run(['cat', file], stdout=subprocess.PIPE)
                f.write(res.stdout.decode()[3:-2] + '\n')
def cutSwarmByNjobs(swarmfile, N=2000):    
    a = []
    with open(swarmfile) as f:
        lines = f.read().splitlines()
        N_split = len(lines)//N
        for i in range(N_split+1):
            fname,ext = swarmfile.split('.')
            newfile='{0}_{1}cut{2}.{3}'.format(fname, str(N), str(i), ext)
            a.append(newfile)
            with open(newfile, 'w') as f:
                f.write('\n'.join(lines[(i*N):((i+1)*N)]))
    return (a)

def submitTerminal(command, printing=False, message=''):
    # quick command to submit jobs to terminal
    start = time.time()
    res=subprocess.run(command.split(' '), stdout=subprocess.PIPE)
    end = time.time()
    sys.stdout.write('EXEC_TIME in sec: '+ str(round(end - start, 3)) + ' : ')
    if printing:
        print(res.stdout.decode('utf-8'))
    if message=='':
        return(res.stdout.decode('utf-8'))
    else:
        print(message, '\n')

In [2]:
!mkdir -p LocusZoom
!mkdir -p LocusCompare

## From rsID to gene names

Couple of eQTL stats to pull
1. eqtlgen site(https://www.eqtlgen.org/)
2. Brain-eQTL (https://www.nature.com/articles/s41467-018-04558-1)
3. Brain regional eQTL (https://doi.org/10.7303/syn2580853, https://www.synapse.org/#!Synapse:syn17015233)

These eqtl files are very large and wouldn't want to load it on python. 

Reference
* 2: Qi, T., Wu, Y., Zeng, J. et al. Identifying gene targets for brain-related traits using transcriptomic and methylomic data from blood. Nat Commun 9, 2282 (2018).
* 3: Allen, M et al. Human whole genome genotype and transcriptome data for Alzheimer's and other neurodegenerative diseases. Sci. Data 3:160089 doi: 10.1038/sdata.2016.89 (2016).https://www.nature.com/articles/sdata201689

In [ ]:
snps=['rs60871478']

In [24]:

# Tissue
tissue='brain'

# Load the target gwas
gwas_path=f'/data/CARD/projects/longGWASnextflow/BiomarkerGWAS-2/METAL/FUMA/log_CSF_pTau.Pi_Deming.tbl.gz'
gwas=pd.read_csv(gwas_path, sep='\t', engine='c')


# get list of genes close to snps
for snp in snps:
    print(snp)
    cmd=f'mkdir -p {snp}/{tissue}'
    
    # brain
    cmd = list()
    eQTLpath = '/data/CARD/OTHER/cortical_meta_eqtl_summary_stats/Cortex_MetaAnalysis_ROSMAP_CMC_HBCC_Mayo_cis_eQTL_release.csv'
    ofile = f'LocusCompare/{snp}/{tissue}/eqtl.txt'
    cmd.append(f"head -n1 {eQTLpath} > {ofile}")
    cmd.append(f"awk 'BEGIN{{FS=\",\"}};{{if($3==\"{snp}\")print}}' {eQTLpath} >> {ofile}")
    cmd = ';'.join(cmd)
    !{cmd}
    
    # Read the output above and standardize the format
    d = pd.read_csv(f'LocusCompare/{snp}/{tissue}/eqtl.txt')
    d = d.rename(columns={'chromosome':'chr', 'snpLocation':'pos'})
    d['tissue'] = tissue
    d=d.rename(columns={'A1':'ref', 'A2':'alt', 'A2freq':'altFreq'}) # Flipped.
    d=d[['snpid', 'chr', 'pos', 'ref', 'alt', 'altFreq', 'tissue', 'gene', 'geneSymbol',  'expressionIncreasingAllele','pvalue', 'FDR',]].copy()
    d.to_csv(f'LocusCompare/{snp}/{tissue}/eqtl_std.csv', index=False)
    
    # Gene list
    genes_interest=d.gene[d.FDR<0.05]
    print(gene_interest)
    
    for gene in genes_interest:
        print(gene, 'processing')
        # get the eqtl sumstats for the gene
        eQTLpath = '/data/CARD/OTHER/cortical_meta_eqtl_summary_stats/Cortex_MetaAnalysis_ROSMAP_CMC_HBCC_Mayo_cis_eQTL_release.csv'
        ofile = f'LocusCompare/{snp}/{tissue}/{gene}_eqtl.txt'
        cmd.append(f"echo 'rsid pval'> {ofile}")
        cmd.append(f"awk 'BEGIN{{FS=\",\"}};{{if($5==\"{gene}\")print $3,$8}}' {eQTLpath} >> {ofile}")
        cmd=(';'.join(cmd))
        !{cmd}

        # Slice the gwas
        d=pd.read_csv(f'LocusCompare/{snp}/{tissue}/{gene}_eqtl.txt', sep=' ')
        dg=gwas[gwas.rsID.isin(d.rsid)].copy()
        dg['rsid']=dg.rsID
        dg['pval']=dg['P-value']
        dg[['rsid', 'pval']].to_csv(f'LocusCompare/{snp}/{tissue}/{gene}_gwas.txt', sep=' ', index=False)
    


In [49]:

# Tissue
tissue='blood'

# Load the target gwas
gwas_path=f'/data/CARD/projects/longGWASnextflow/BiomarkerGWAS-2/METAL/FUMA/log_CSF_pTau.Pi_Deming.tbl.gz'
gwas=pd.read_csv(gwas_path, sep='\t', engine='c')
gwas=gwas[(gwas.OBS_CT_TOTAL>3000)&(gwas.HetISq<80)].copy()

# get list of genes close to snps
for snp in snps:
    print(snp)
    cmd=f'mkdir -p {snp}/{tissue}'
    !{cmd}
    
    # blood
    cmd = list()
    eQTLpath = '/data/CARD/OTHER/eqtlgen_summary_stats/2019-12-11-cis-eQTLsFDR-ProbeLevel-CohortInfoRemoved-BonferroniAdded.txt.gz'
    ofile = f'LocusCompare/{snp}_blood_eqtl.txt'
    cmd.append(f"zcat {eQTLpath} | head -n1 > {ofile}")
    cmd.append(f"zcat {eQTLpath} | awk '{{if($2==\"{snp}\")print}}' >> {ofile}")
    cmd = ';'.join(cmd)
#     !{cmd}
    
    # Read the output above and standardize the format
    d = pd.read_csv(f'LocusCompare/{snp}/{tissue}/eqtl.txt', sep='\t')
    d = d.rename(columns={'chromosome':'chr', 'snpLocation':'pos'})
    d['tissue'] = tissue
    d['expressionIncreasingAllele'] = d.AssessedAllele.where(d.Zscore>0, d.OtherAllele)
    d=d.rename(columns={'AssessedAllele':'ref', 'OtherAllele':'alt', 
                        'SNP':'snpid', 'SNPChr':'chr', 'SNPPos':'pos', 
                        'Pvalue':'pvalue', 'Gene':'gene', 'GeneSymbol':'geneSymbol'}) # Flipped.
    d['altFreq']=np.nan
    d=d[['snpid', 'chr', 'pos', 'ref', 'alt', 'altFreq', 'tissue', 'gene', 'geneSymbol',  'expressionIncreasingAllele','pvalue', 'FDR',]].copy()
    d.to_csv(f'LocusCompare/{snp}/{tissue}/eqtl_std.csv', index=False)
    
    # Gene list
    genes_interest=d.gene[d.FDR<0.05]
    print(gene_interest)
    
    for gene in genes:
        print(gene, 'processing')
        # get the eqtl sumstats for the gene
        cmd = list()
        eQTLpath = '/data/CARD/OTHER/eqtlgen_summary_stats/2019-12-11-cis-eQTLsFDR-ProbeLevel-CohortInfoRemoved-BonferroniAdded.txt.gz'
        ofile = f'LocusCompare/eQTL/{gene}_blood_eqtl.txt'
        cmd.append(f"echo 'rsid pval'> {ofile}")
        cmd.append(f"zcat {eQTLpath} | awk '{{if($8==\"{gene}\")print $2,$1}}' >> {ofile}")
        cmd=';'.join(cmd)
        !{cmd}

        # Slice the gwas
        d=pd.read_csv(f'LocusCompare/{snp}/{tissue}/{gene}_eqtl.txt', sep=' ')
        dg=gwas[gwas.rsID.isin(d.rsid)].copy()
        dg['rsid']=dg.rsID
        dg['pval']=dg['P-value']
        dg[['rsid', 'pval']].to_csv(f'LocusCompare/{snp}/{tissue}/{gene}_gwas.txt', sep=' ', index=False)

In [ ]:
ENSG00000240230_eqtl X
ENSG00000239857_eqtl X
ENSG00000188191_eqtl ?
ENSG00000164849_eqtl X
ENSG00000164818_eqtl O
ENSG00000146540_eqtl X
ENSG00000105963_eqtl X

# LocusCompare browser is useful to check for a smaller number of comparison